# Построение baseline моделей

## 1. Подготовка

Импорт необходимых библиотек

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, Imputer, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, classification_report, roc_curve

Загрузка данных:

In [22]:
X = pd.read_csv('F:\\Work\\My\\Python\\courses\\courserra\\ml-specialization\\materials\\course6\\churn\\orange_small_churn_data.txt')
y = pd.read_csv('F:\\Work\\My\\Python\\courses\\courserra\\ml-specialization\\materials\\course6\\churn\\orange_small_churn_labels.txt', header=None, names=['target'])
y['target'] = y['target'].apply(lambda x: 1 if x==1 else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=9,
                                                    shuffle=True,
                                                    stratify=y)

X_train = X_train.iloc[:10000, :]
y_train = y_train.iloc[:10000, :].values.ravel()

В выборке всего 32000 параметров, что приводит к значительному времени обучения моделей. Ограничим выборку до 10000. Также временно будем кросс-валидироваться на 5 фолдах, а не на 7. Это снизит время обучения и не слишком скажется на качестве baseline моделей.

Для начала реализуем препроцессинг данных.

Данные содержат много пропусков и выбросов. С выбросами пока ничего делать не будем, а пропуски заполним средними значениями для случая числовых признаков и отдельным признаком для случая категориальных признаков. Кроме того, у некоторых категориальных признаков большое число возможных значений. Ограничим его, взяв токо первые 200 наиболее часто встречаемых значения для каждого категориального признака.

Нам потребуется отдельный класс, который будет делать dummy-encoding категориальных переменных с указанными выше особенностями

In [23]:
class DummyEncoder(BaseEstimator, TransformerMixin):
    '''
    Encodes categorical features as one-hot variables with max_categories restriction
    '''
    def __init__(self, columns=None, max_categories=None):
        self.columns = columns
        self.dummy_columns = None
        self.max_categories = max_categories


    def fit(self, X, y=None, **kwargs):
        self.dummy_columns = None
        return self


    def transform(self, X, y=None, **kwargs):
        if self.max_categories is not None:
            X = X[self.columns] if self.columns is not None else X.copy()
            for col in X.columns:
                top_cats = X[col].value_counts()[:self.max_categories].index.values
                X[col] = X[col].apply(lambda x: x if (x in top_cats or x is None) else 'aggr')

        dummy_df = pd.get_dummies(X, columns=self.columns, sparse=True, dummy_na=True)
        new_cols = dummy_df.columns.values
        if self.dummy_columns is None:
            self.dummy_columns = new_cols
            return dummy_df
        else:
            res_df = pd.DataFrame()
            for col in self.dummy_columns:
                res_df[col] = dummy_df[col] if col in new_cols else np.zeros((len(X),), dtype=int)
        return res_df

Так как будет использоваться кросс-валидация, даже baseline модели нужно тренировать в пайплайне, чтобы информация о holdout-выборках не использовалась при обучении. Пайплайн будет:
- Удалять полностью NaN признаки
- Запонять средними значенийми пропуски числовых переменных
- Масштабировать числовые переменные
- Dummy-кодировать категориальные переменные по описанному выше алгоритму

Реализуем функцию, конструирующую пайплайн:

In [24]:
def get_baseline_pipeline(X, alg):

    # get non-NaN columns
    vc = X.apply(lambda col: len(col.value_counts()))
    vc = vc[vc > 0]
    all_cols = X.columns.values
    num_cols = set(all_cols[:190])
    cat_cols = set(all_cols[190:])
    non_nan_all_cols = set(vc.index.values)
    non_nan_num_cols = sorted(list(non_nan_all_cols.intersection(num_cols)))
    non_nan_cat_cols = sorted(list(non_nan_all_cols.intersection(cat_cols)))


    pipeline = Pipeline(steps=[
            # get rid of fully NaN columns
            ('filter_out_useless_columns', FunctionTransformer(lambda data: data.loc[:, non_nan_all_cols], validate=False)),

            # processing
            ('processing', FeatureUnion([

                # numeric features
                ('numeric', Pipeline(steps=[
                    ('selecting', FunctionTransformer(lambda data: data.loc[:, non_nan_num_cols], validate=False)),
                    ('float_nan_mean', Imputer(strategy='mean')),
                    ('scaling', StandardScaler())
                ])),

                # categorical features
                ('categorical', Pipeline(steps=[
                    ('selecting', FunctionTransformer(lambda data: data.loc[:, non_nan_cat_cols], validate=False)),
                    ('encoding', DummyEncoder(max_categories=200))
                ]))
            ])),

            #model
            ('model', alg)
        ])

    return pipeline

В качестве основной метрики выберем ROC-AUC метрику. В качестве дополнительных: F1, Precision и Recall.

In [25]:
metrics = [ 
    ('roc_auc',   roc_auc_score), 
    ('precision', precision_score), 
    ('recall',    recall_score), 
    ('f1',        f1_score) 
]

In [45]:
model_results = pd.DataFrame(data=np.zeros((3, 5)), columns=['model'] + [m[0] for m in metrics])


def run_model(model, params, n_folds):
    print('*'*64)
    print(' Running model: {0} '.format(type(model).__name__).center(64, '*'))
    print('*'*64)

    pipeline = get_baseline_pipeline(X_train, model)

    cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=9)

    results = []
    
    for metric, score_func in metrics:
        print('Metric: {0}'.format(metric).center(64))

        grid = GridSearchCV(pipeline, params, scoring=metric, cv=cv, verbose=10)
        grid.fit(X_train, y_train)

        cv_score = grid.best_score_
        
        y_pred = grid.predict_proba(X_test)[:, 1]
        
        fpr, tpr, thr = roc_curve(y_test, y_pred)
        threshold = get_optimal_threshold(fpr, tpr, thr)
        print('optimal threshold:', threshold)
        if metric != 'roc_auc':
            y_pred = y_pred > threshold

        ho_score = score_func(y_test, y_pred)

        results.append(ho_score)
        
        print('Metric {0} results:'.format(metric))
        print('CV score:', cv_score)
        print('Holdout score:', ho_score)
        print('Report:')
        print('Best params:', grid.best_params_)
        print(classification_report(y_test, y_pred>threshold))

    print('\n-------------------------------------\n')

    return results


def get_optimal_threshold(fprs, tprs, thrs):
    n = len(fprs)
    dist = 10
    thr_opt = None
    for i in range(n):
        fpr, tpr, thr = fprs[i], tprs[i], thrs[i]
        d = fpr*fpr + (tpr - 1)*(tpr - 1)
        if d < dist:
            thr_opt = thr
            dist = d
    return thr_opt

Можно приступать к построению и обучению моделей. В качестве baseline моделей будем использовать
- Случайный лес
- Логистическую регрессию
- Градиентный бустинг

Наилучшие параметры алгоритмов выбираются GridSearch-ем

## 2. Случайный лес

In [46]:
model = RandomForestClassifier(random_state=9, n_jobs=-1)
params = { 'model__n_estimators': [100, 150, 200] }
results = run_model(model, params, 5)

model_results.loc[0] = np.array(['RandomForestClassifier', *results])

****************************************************************
************ Running model: RandomForestClassifier *************
****************************************************************
                        Metric: roc_auc                         
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] model__n_estimators=100 .........................................
[CV]  model__n_estimators=100, score=0.6049992352188666, total=   8.5s
[CV] model__n_estimators=100 .........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.5s remaining:    0.0s


[CV]  model__n_estimators=100, score=0.6132345375159909, total=   8.4s
[CV] model__n_estimators=100 .........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   27.0s remaining:    0.0s


[CV]  model__n_estimators=100, score=0.6266460175760611, total=   8.5s
[CV] model__n_estimators=100 .........................................


...

[CV] ............... model__n_estimators=200, score=0.0, total=  10.2s
[CV] model__n_estimators=200 .........................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ............... model__n_estimators=200, score=0.0, total=  10.0s


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.6min finished


optimal threshold: 0.08
Metric f1 results:
CV score: 0.0
Holdout score: 0.198606271777
Report:
Best params: {'model__n_estimators': 100}
             precision    recall  f1-score   support

          0       0.95      0.73      0.82      7405
          1       0.13      0.48      0.20       595

avg / total       0.88      0.71      0.78      8000


-------------------------------------



Результаты:

In [51]:
model_results.iloc[0, :]

model        RandomForestClassifier
roc_auc              0.655579865977
precision            0.125274725275
recall               0.478991596639
f1                   0.198606271777
Name: 0, dtype: object

## 3. Логистическая регрессия


In [52]:
model = LogisticRegression(random_state=9, n_jobs=-1)
params = { 'model__C': [0.1, 1, 10] }
results = run_model(model, params, 5)

model_results.loc[1] = np.array(['LogisticRegression', *results])

****************************************************************
************** Running model: LogisticRegression ***************
****************************************************************
                        Metric: roc_auc                         
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] model__C=0.1 ....................................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


[CV] ........... model__C=0.1, score=0.6201262584125925, total=   7.5s
[CV] model__C=0.1 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.3s remaining:    0.0s
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


[CV] ........... model__C=0.1, score=0.6528693197619444, total=   7.3s
[CV] model__C=0.1 ....................................................


...

[CV] ........................ model__C=10, score=0.0625, total=  11.7s
[CV] model__C=10 .....................................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


[CV] ........... model__C=10, score=0.08205128205128205, total=   9.7s


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.6min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


optimal threshold: 0.05084752634
Metric f1 results:
CV score: 0.104921781063
Holdout score: 0.173464603844
Report:
Best params: {'model__C': 10}
             precision    recall  f1-score   support

          0       0.95      0.55      0.70      7405
          1       0.10      0.62      0.17       595

avg / total       0.89      0.56      0.66      8000


-------------------------------------



Результаты

In [55]:
model_results.iloc[1, :]

model        LogisticRegression
roc_auc          0.650846180471
precision        0.101038450744
recall           0.621848739496
f1               0.173464603844
Name: 1, dtype: object

## 4. Градиентный бустинг

In [56]:
model = GradientBoostingClassifier(learning_rate=0.1, random_state=9)
params = { 'model__n_estimators': [100] }
results = run_model(model, params, 5)

model_results.loc[2] = np.array(['GradientBoostingClassifier', *results])

****************************************************************
********** Running model: GradientBoostingClassifier ***********
****************************************************************
                        Metric: roc_auc                         
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] model__n_estimators=100 .........................................
[CV]  model__n_estimators=100, score=0.7090233744924634, total=  34.7s
[CV] model__n_estimators=100 .........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   39.8s remaining:    0.0s


[CV]  model__n_estimators=100, score=0.7050986567662273, total=  34.0s
[CV] model__n_estimators=100 .........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s


[CV]  model__n_estimators=100, score=0.7270913287724569, total=  33.8s
[CV] model__n_estimators=100 .........................................


...

[CV] ............... model__n_estimators=100, score=0.0, total=  33.9s
[CV] model__n_estimators=100 .........................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.6min remaining:    0.0s


[CV]  model__n_estimators=100, score=0.03614457831325302, total=  33.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.2min finished


optimal threshold: 0.0845452355231
Metric f1 results:
CV score: 0.0245138106823
Holdout score: 0.24037184595
Report:
Best params: {'model__n_estimators': 100}
             precision    recall  f1-score   support

          0       0.96      0.72      0.82      7405
          1       0.15      0.61      0.24       595

avg / total       0.90      0.71      0.78      8000


-------------------------------------



Результаты:

In [57]:
model_results.iloc[2, :]

model        GradientBoostingClassifier
roc_auc                  0.723944870318
precision                 0.14977244518
recall                   0.608403361345
f1                        0.24037184595
Name: 2, dtype: object

## 5. Результаты

In [58]:
model_results

,model,roc_auc,precision,recall,f1
0,RandomForestClassifier,0.655579865977,0.125274725275,0.478991596639,0.198606271777
1,LogisticRegression,0.650846180471,0.101038450744,0.621848739496,0.173464603844
2,GradientBoostingClassifier,0.723944870318,0.14977244518,0.608403361345,0.24037184595
